##CREATE FACT TABLE

***Reading Silver Data***


In [0]:

df_silver=spark.sql('''
        SELECT * 
        from parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`
        ''')

In [0]:
df_silver.display()

***rading all the dimesions 

In [0]:
df_dealer=spark.sql('''
                    select * from cars_catalog.gold.dim_dealer
                    ''')
df_branch=spark.sql('''
                    select * from cars_catalog.gold.dim_branch
                    ''')
df_date=spark.sql('''
                    select * from cars_catalog.gold.dim_date
                    ''')
df_model=spark.sql('''
                    select * from cars_catalog.gold.dim_model
                    ''')

***Bringing Keys to the fact table***

In [0]:
df_fact=df_silver.join(df_branch,df_silver['Branch_id']==df_branch['Branch_Id'],'left')\
                 .join(df_date,df_silver['Date_Id']==df_date['Date_Id'],'left')\
                 .join(df_dealer,df_silver['Dealer_Id']==df_dealer['Dealer_Id'],'left')\
                 .join(df_model,df_silver['Model_Id']==df_model['Model_Id'],'left')\
                .select(df_silver['Revenue'],df_silver['Units_Sold'],df_silver['RevenuePerUnits'],df_branch['dimi_branch_key'],df_dealer['dimi_dealer_key'],df_date['dimi_date_key'],df_model['dim_model_key'])

In [0]:
df_fact.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('fact_sales'):
    delta_table = DeltaTable.forPath(spark, 'cars_catalog.gold.fact_sales')

    delta_table.alias('trg').merge(
        df_fact.alias('src'),
        'trg.dimi_branch_key = src.dimi_branch_key AND trg.dimi_dealer_key = src.dimi_dealer_key AND trg.dimi_date_key = src.dimi_date_key AND trg.dim_model_key = src.dim_model_key'
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

else:
    # Save data using the external location you defined
    df_fact.write.format('delta') \
        .mode('overwrite') \
        .option('path', 'abfss://gold@carkrishdatalake.dfs.core.windows.net/facts_sales') \
        .saveAsTable('cars_catalog.gold.fact_sales')


In [0]:
%sql
select * from cars_catalog.gold.fact_sales